# Setup environment

In [1]:
import datetime
from dotenv import load_dotenv, find_dotenv
import os
from typing import List, Dict, Any

In [2]:
# https://pypi.org/project/openai/
import openai
from openai.openai_object import OpenAIObject

In [3]:
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY', 'MISSING_OPENAI_API_KEY')

# Prompt LLM

## Create helpers

In [4]:
def get_completion(prompt: str, model: str, temperature: float, n_completions: int) -> str:
    messages: List[Dict[str, Any]] = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    try:
        response: OpenAIObject = openai.ChatCompletion.create(
            messages=messages,
            model=model,
            temperature=temperature,
            n=n_completions
        ) 
    except Exception as e:
        return f"Error: {e}"

    if not response.choices:
        return "Error: No completions returned"

    return response.choices[0].message.content

## Set prompting params

In [5]:
# https://platform.openai.com/docs/models/model-endpoint-compatibility
completion_model = 'gpt-3.5-turbo'

In [6]:
# https://platform.openai.com/docs/api-reference/completions/create#completions/create-temperature
temperature = 0.01

In [7]:
# https://platform.openai.com/docs/api-reference/completions/create#completions/create-n
n_completions = 1

In [8]:
prompting_params: Dict[str, Any] = {
    "model": completion_model,
    "temperature": temperature,
    "n_completions": n_completions
}

## Start prompting

### Prompting Principle 1: Write clear and specific instructions

Tactics in the principle:
1. Use delimiters to clearly indicate distinct parts of the input
2. Ask for a structured output
3. Ask the model to check whether conditions are satisfied
4. "Few-shot" prompting

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [24]:
text = f"""
Prom.ua — український маркетплейс, проєкт IT-компанії EVO.
На його платформі підприємці[1] самостійно створюють інтернет-магазини і / або розміщують свої товари в загальному каталозі.
Для покупців на Промі зібрано більше 100 мільйонів товарів.
В 2018 році українці витратили на маркетплейсі 8,6 млрд грн — і ще 10,5 млрд грн на сайтах компаній, створених на базі Prom.ua[3].
Щодня торговельний майданчик відвідує близько 2 млн користувачів.
У ТОП-10 популярних категорій входять одяг, взуття, аксесуари, товари для дому та саду, краси та здоров'я, а також техніка й електроніка[4].
Щомісяця на маркетплейсі покупці оформлюють у середньому 1,5 млн замовлень на суму близько 1,3 млрд грн[5].
"""
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
The summarized text must be written in Ukrainian.

```{text}```
"""
response = get_completion(prompt, **prompting_params)
print(response)

Prom.ua - український маркетплейс, на якому підприємці можуть створювати інтернет-магазини та розміщувати свої товари, а покупці можуть знайти більше 100 мільйонів товарів, що призвело до витрат українців на маркетплейсі 8,6 млрд грн у 2018 році, а також до оформлення щомісяця близько 1,5 млн замовлень на суму близько 1,3 млрд грн.


#### Tactic 2: Ask for a structured output
- use JSON, HTML

In [23]:
prompt = f"""
Generate a list of top 5 modern Ukrainian e-commerce marketplaces.

Provide them in JSON format with the following keys: 
business_id, name, users, user_rating, free_delivery, has_android_app, has_ios_app
"""
response = get_completion(prompt, **prompting_params)
print(response)

[
  {
    "business_id": 1,
    "name": "Rozetka",
    "users": 10,000,000,
    "user_rating": 4.5,
    "free_delivery": true,
    "has_android_app": true,
    "has_ios_app": true
  },
  {
    "business_id": 2,
    "name": "Prom.ua",
    "users": 5,000,000,
    "user_rating": 4.2,
    "free_delivery": false,
    "has_android_app": true,
    "has_ios_app": true
  },
  {
    "business_id": 3,
    "name": "Epicentrk",
    "users": 3,000,000,
    "user_rating": 4.0,
    "free_delivery": true,
    "has_android_app": true,
    "has_ios_app": true
  },
  {
    "business_id": 4,
    "name": "Allo",
    "users": 2,500,000,
    "user_rating": 4.1,
    "free_delivery": true,
    "has_android_app": true,
    "has_ios_app": true
  },
  {
    "business_id": 5,
    "name": "Foxtrot",
    "users": 2,000,000,
    "user_rating": 4.3,
    "free_delivery": false,
    "has_android_app": true,
    "has_ios_app": true
  }
]


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [25]:
text = f"""
Hello! My name is Max and I am writing a text for Tactic 3 section.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 

If it contains a word ROZETKA, re-write the text, replacing each vowel with ROZETKA.
If the text does not contain a word ROZETKA, then simply write \"No ROZETKA for now\"

\"\"\"{text}\"\"\"
"""
response = get_completion(prompt, **prompting_params)
print(response)

No ROZETKA for now


In [26]:
text = f"""
Hello! My name is Max and I am writing a text for Tactic 3 section.ROZETKA
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 

If it contains a word ROZETKA, re-write the text, replacing each vowel with ROZETKA.
If the text does not contain a word ROZETKA, then simply write \"No ROZETKA for now\"

\"\"\"{text}\"\"\"
"""
response = get_completion(prompt, **prompting_params)
print(response)

Horozerlo! My naroze is Mrozex rozend I am wroziting a text for Tacticrozeka 3 secrozetion.


#### Tactic 4: "Few-shot" prompting

In [28]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about e-commerce business.

<pirate>: Ahoy, matey! Ye be wantin' to learn about e-commerce, do ye? \
'Tis a way of buying and selling goods and services over the interwebs, \
without needin' to set sail to a port. To start an e-commerce business, \
ye need to create an online store and list yer wares. Use digital marketing \
to attract customers and offer secure payment options and fast shipping \
to keep 'em coming back for more. With a bit of savvy and know-how, ye can \
have a thriving online business, arrr!

<child>: Teach me about Ukrainian e-commerce.
"""
response = get_completion(prompt, **prompting_params)
print(response)

<pirate>: Avast, young one! Ye be curious about e-commerce in Ukraine, eh? Well, 'tis similar to e-commerce in other parts of the world, but with its own unique flavor. Ukrainian e-commerce businesses offer a variety of products and services, from handmade crafts to high-tech gadgets. To succeed in this market, ye need to understand the local culture and preferences, as well as the legal and logistical challenges of operating in Ukraine. But fear not, with determination and a bit of luck, ye can hoist the Jolly Roger and set sail for e-commerce success in Ukraine! Arrr!


### Prompting Principle 2: Give the model time to “think”

Tactics in the principle:
1. Specify the steps required to complete a task
2. Instruct the model to work out its own solution before rushing to a conclusion

#### Tactic 1: Specify the steps required to complete a task

In [30]:
text = f"""
Усі 15 років піклуємося про твій час, тому на Промчику ти можеш знайти все необхідне, а ми постійно вдосконалюємо процеси замовлення.

Сьогодні приймаємо вітання, але також даруємо тобі можливість виграти смарт-годинник Gelius Pro GP-SW004🥳

Щоб взяти участь у розіграші, потрібно:
✅ Підписатися на @prom.pompom
✅ Тегнути у коментарях друга

Чим більше тегів друзів, тим більше шансів отримати приз😊

Результат оголосимо в сторіз вже 5.05 за допомогою рандомайзера.

Офіційні умови розіграшу закріпили за посиланням у шапці профілю.
Щасти 🤞🏻
"""

In [31]:
# example 1
prompt = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple backticks with 1 sentence.
2 - Translate the summary into Polish.
3 - List each online account name in the Polish summary.
4 - Output a json object that contains the following \
keys: polish_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt, **prompting_params)
print(response)

Summary: Promchik is celebrating its 15th anniversary by offering a chance to win a Gelius Pro GP-SW004 smartwatch by subscribing to @prom.pompom and tagging a friend in the comments, with more tags increasing the chances of winning.

Polish summary: Promchik obchodzi swoje 15-lecie, oferując szansę wygrania smartwatcha Gelius Pro GP-SW004 poprzez subskrypcję @prom.pompom i oznaczenie przyjaciela w komentarzach, a większa liczba oznaczeń zwiększa szanse na wygraną.

Online account names: @prom.pompom

JSON object: {"polish_summary": "Promchik obchodzi swoje 15-lecie, oferując szansę wygrania smartwatcha Gelius Pro GP-SW004 poprzez subskrypcję @prom.pompom i oznaczenie przyjaciela w komentarzach, a większa liczba oznaczeń zwiększa szanse na wygraną.", "num_names": 1}


In [32]:
# example 2
prompt = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by <> with 1 sentence.
2 - Translate the summary into English.
3 - List each name in the English summary.
4 - Output a json object that contains the following keys: english_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt, **prompting_params)
# pprint.pprint(response)
print(response)

Summary: Promchik is celebrating its 15th anniversary and giving away a Gelius Pro GP-SW004 smartwatch to participants who follow their Instagram account and tag a friend in the comments, with more tags increasing the chances of winning. 
Translation: Promchik is celebrating its 15th anniversary and giving away a Gelius Pro GP-SW004 smartwatch to participants who follow their Instagram account and tag a friend in the comments, with more tags increasing the chances of winning.
Names: Promchik
Output JSON: {"english_summary": "Promchik is celebrating its 15th anniversary and giving away a Gelius Pro GP-SW004 smartwatch to participants who follow their Instagram account and tag a friend in the comments, with more tags increasing the chances of winning.", "num_names": 1}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [33]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt, **prompting_params)
print(response)

The student's solution is correct.


In [34]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt, **prompting_params)
print(response)

The student's solution is correct.


### Model limitations: Hallucinations

In [35]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Quick Payment service by prom.ua
"""
response = get_completion(prompt, **prompting_params)
print(response)

As an AI language model, I don't have access to the latest updates or news. However, I can provide you with general information about AeroGlide UltraSlim Smart Quick Payment service by prom.ua.

AeroGlide UltraSlim Smart Quick Payment service is a payment solution offered by prom.ua, a Ukrainian e-commerce platform. This service allows customers to make quick and secure payments for their online purchases using their smartphones. The payment process is streamlined and easy to use, with no need for customers to enter their payment details manually.

The service uses NFC (Near Field Communication) technology to enable contactless payments. Customers simply need to hold their smartphones close to the payment terminal to complete the transaction. The service is compatible with most smartphones and payment terminals, making it accessible to a wide range of customers.

AeroGlide UltraSlim Smart Quick Payment service is designed to provide a fast and convenient payment experience for customer

In [33]:
print(datetime.datetime.now())

2023-05-01 16:08:12.193340
